In [1]:
# this file will be used to load the data from the processed_data directory into the chroma database
import chromadb
from langchain_openai import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_chroma import Chroma
from langchain.storage import LocalFileStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
import uuid
import os
from dotenv import load_dotenv
load_dotenv()


CHROMA_PATH = "../data/db/chroma"
DOCSTORE_PATH = "../data/db/docstore"

os.makedirs(CHROMA_PATH, exist_ok=True)

# chroma DB
client = chromadb.PersistentClient(CHROMA_PATH)
# vector_store = Chroma(
#     client=client,
#     embedding_function=OpenAIEmbeddings(),
#     collection_name="softeon"
# )

# # document store
# docstore = LocalFileStore(DOCSTORE_PATH)
# # basically what the metadata key for the ID of the document will be called
# id_key = "doc_id"

# # multi vector retriever
# retriever = MultiVectorRetriever(
#     vectorstore=vector_store,
#     docstore=docstore,
#     id_key=id_key
# )

In [2]:
#Self-Query Retreiver
self_query_store = Chroma(
    client=client,
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"),
    collection_name="softeon_metadata_v3"
)

metadata_field_info = [
    AttributeInfo(
        name="file_name",
        description="The name of the file. You must only use this attribute if the filename is mentioned fully by the user in the query.",
        type="string",
    ),
    AttributeInfo(
        name="tag",
        description="The tag that is most appropriate for the text content. Please use the tags exactly as shown. One of the following: [WMS, Administrative, User Interface, Operational Processes, Automation, Inbound, Outbound, Integration, Compliance]",
        type="string",
    ),
    AttributeInfo(
        name="file_type",
        description="The type of the file. One of the following: [text, image]. `image` refers to the screens of the application.",
        type="string",
    ),
    AttributeInfo(
        name="page_number",
        description="The page number of the document. Please use this attribute only if the page number is mentioned in the query.",
        type="int",
    ),
    AttributeInfo(
        name="image_number",
        description="The image number of the document. Please use this attribute only if the image number is mentioned in the query and only for image `file_type`.",
        type="int",
    ),
]

document_content_description = "The documents contain information about a warehouse management system called Softeon."

llm = ChatOpenAI(model="gpt-4o-2024-08-06")

In [30]:
self_query_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=self_query_store,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    search_kwargs={"k": 10},
)

In [2]:
def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))

In [ ]:
query = "How to create new user"
docs = retriever.invoke(input="How to create a new user", limit=10)

In [3]:
query = "How to create new user"
docs = self_query_retriever.invoke(input=query, limit=10)

In [4]:
docs

[Document(metadata={'file_name': 'SFTN_WMS_SecurityProfile-UserGuide.txt', 'file_type': 'image', 'image_number': 2, 'page_number': 8, 'tag': 'Administrative'}, page_content='The provided image is a screenshot of a user interface from a software application named Softeon. The screen appears to be for copying user details or creating a new user based on an existing one.\n\nHere is a detailed breakdown of the elements in the image:\n\n1. **Title and Branding**:\n   - The top-left corner features the Softeon logo.\n   - The title of the screen is "Copy From User".\n\n2. **User Information Section**:\n   - This section contains fields for entering user details.\n   - **User ID**: Already populated with "ATMUSER1".\n   - **User Name**: Populated with "Automation User".\n   - **User Group**: Populated with "S7UG1".\n\n3. **Input Fields**:\n   - **First Name**: An empty text field marked with a red asterisk, indicating it is required.\n   - **Last Name**: An empty text field marked with a red 

In [5]:
q2 = "What are some administrative tasks that can be performed in the WMS software?"
docs2 = self_query_retriever.invoke(input=q2, limit=5)

In [6]:
docs2

[Document(metadata={'file_name': 'SFTN_WMS_SecurityProfile-UserGuide.txt', 'file_type': 'image', 'image_number': 2, 'page_number': 16, 'tag': 'WMS'}, page_content='The image displays a screenshot of a "Security Profile" interface from a software application named "Softeon." The interface appears to be used for managing user access within a warehouse management system (WMS). Here’s a detailed summary:\n\n### Interface Components:\n1. **Navigation Tabs:**\n   - "User"\n   - "Group"\n   - "Advanced User Map"\n   - "Module Map"\n   - "User Access Map"\n   - "Menu Access"\n\n2. **Filters and Fields:**\n   - **Organization (ORG1)**\n   - **App ID (WMS)**\n   - **Warehouse (WH1)**\n   - **Bldg ID** (Building ID)\n   - **User Group (ADM)**\n   - **Business Unit**\n   - **User ID**\n\n3. **Buttons:**\n   - **Submit** (highlighted with a red border)\n   - **Reset**\n   - **Find**\n   - **Refresh**\n   - **Copy**\n\n4. **Table:**\n   - Contains columns for:\n     - **App ID**\n     - **Warehouse 

In [31]:
q3 = "Summarize the document: SFTIN_WMS_AdhocMasterTemplate-UserGuide.txt"
docs3 = self_query_retriever.invoke(input=q3)

In [32]:
docs3

[Document(metadata={'file_name': 'SFTIN_WMS_AdhocMasterTemplate-UserGuide.txt', 'file_type': 'image', 'image_number': 3, 'page_number': 1, 'tag': 'Operational Processes'}, page_content='The image depicts a scene from a warehouse. A worker, who is wearing a hard hat and a high-visibility safety vest, is seen from the back as they operate a manual pallet jack. The worker appears to be moving through a narrow aisle lined with tall shelves stacked with various items, suggesting an organized storage system. The background is blurred, giving a sense of motion and speed, indicating that the worker is actively moving through the aisle. The perspective and the blur effect emphasize the depth and scale of the warehouse. The overall scene portrays a dynamic and industrious environment, typical of a logistics or storage facility.'),
 Document(metadata={'file_name': 'SFTIN_WMS_AdhocMasterTemplate-UserGuide.txt', 'file_type': 'image', 'image_number': 4, 'page_number': 1, 'tag': 'Operational Processe